In [1]:
%pwd

'c:\\Users\\saibh\\OneDrive\\Documents\\projects\\GenAiProjects\\Medical-ChatBot\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\saibh\\OneDrive\\Documents\\projects\\GenAiProjects\\Medical-ChatBot'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

C:\Users\saibh\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
# extracted_data

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [11]:
# text_chunks

In [12]:
%pip install -U sentence-transformers

from langchain_huggingface import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()
print("Embeddings loaded successfully ✅")


Note: you may need to restart the kernel to use updated packages.
Embeddings loaded successfully ✅


In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [14]:
# query_result

In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [17]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=384, 
        metric="cosine", 
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"
        ) 
    )
else:
    print(f"Index '{index_name}' already exists. Skipping creation.")

In [18]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [19]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

c:\Users\saibh\anaconda3\envs\pro\lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [20]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [21]:
docsearch

In [22]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [23]:
retrieved_docs = retriever.invoke("What is Acne?")

In [24]:
retrieved_docs

[Document(id='099cfaec-3a70-4996-bb89-6cbfea1721ad', metadata={'page': 39.0, 'page_label': '40', 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='2a14c804-6566-490b-978c-6bf6f9272a50', metadata={'page': 38.0, 'page_label': '39', 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='0e7187f5-108b-4829-801c-c890cce6454b', metadata={'page': 37.0, 'page_label': '38', 'source': 'Data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease character

In [25]:
pip install huggingface-hub


In [26]:
pip install hf_xet

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.8 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.8 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.8 MB 621.2 kB/s eta 0:00:04
   ----------- ---------------------------- 0.8/2.8 MB 799.2 kB/s eta 0:00:03
   ------------------ --------------------- 1.3/2.8 MB 1.1 MB/s eta 0:00:02
   ---------------------- ----------------- 1.6/2.8 MB 1.2 MB/s eta 0:00:02
   ----------------------------- ---------- 2.1/2.8 MB 1.4 MB/s eta 0:00:01
   ------------------------------------- -- 2.6/2.8 MB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 1.6 MB/s  0:00:01
Note: you may need to restart the kernel to use updated packages.


In [29]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline

# Hugging Face CLI already handles authentication
generator = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.2",
    tokenizer="mistralai/Mistral-7B-Instruct-v0.2",
    max_new_tokens=500,
    temperature=0.4
)

llm = HuggingFacePipeline(pipeline=generator)


Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.84it/s]
Device set to use cpu


In [30]:
# Prompt Template

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following retrieved context to answer the question. "
    "If you don't know the answer, just say you don't know. "
    "Keep the answer concise, within three sentences.\n\n"
    "{context}"
)


In [32]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)



In [33]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Retrieval Augmented Generation (RAG)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [34]:

# ------------------------
# Test Queries
# ------------------------
response = rag_chain.invoke({"input": "What is Acromegaly and gigantism?"})
print("Q1:", response["answer"])



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q1: System: You are an assistant for question-answering tasks. Use the following retrieved context to answer the question. If you don't know the answer, just say you don't know. Keep the answer concise, within three sentences.

Whitehouse Station, NJ: Merck Research Laboratories,
1997.
Larsen, D. E., ed. Mayo Clinic Family Health Book.New York:
William Morrow and Co., Inc., 1996.
John T. Lohr, PhD
Acromegaly and gigantism
Definition
Acromegaly is a disorder in which the abnormal
release of a particular chemical from the pituitary gland
in the brain causes increased growth in bone and soft tis-
sue, as well as a variety of other disturbances throughout
the body. This chemical released from the pituitary gland

mone Excess: Acromegaly and Gigantism.” In Harrison’s
Principles of Internal Medicine, ed. Anthony S. Fauci, et
al. New York: McGraw-Hill, 1997.
Jameson, J. Larry. “Growth Hormone Excess: Acromegaly and
Gigantism.” In Cecil Textbook of Medicine, ed. J. Claude
Bennett and Fred Plum

In [35]:
response = rag_chain.invoke({"input": "What is stats?"})
print("Q2:", response["answer"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q2: System: You are an assistant for question-answering tasks. Use the following retrieved context to answer the question. If you don't know the answer, just say you don't know. Keep the answer concise, within three sentences.

that is often done as part of a routine physical examina-
tion. It can provide valuable information about the blood
and blood-forming tissues (especially the bone marrow),
as well as other body systems. Abnormal results can indi-
cate the presence of a variety of conditions—including
anemias, leukemias, and infections—sometimes before
the patient experiences symptoms of the disease.
Description
A complete blood count is actually a series of tests

Blood count
Definition
One of the most commonly ordered clinical labora-
tory tests, a blood count, also called a complete blood
count (CBC), is a basic evaluation of the cells (red blood
cells, white blood cells, and platelets) suspended in the
liquid part of the blood (plasma). It involves determining
the numbers, co